In [6]:
import numpy as np
import os

In [3]:
from intdielec.io.cp2k import Cp2kOutput, Cp2kHartreeCube, Cp2kCube

In [4]:
class Cp2kTest:
    def __init__(self, work_dir):
        self.work_dir = work_dir
        self.stdout = Cp2kOutput(os.path.join(work_dir, "output.out"))
        self.hartree_cube = Cp2kHartreeCube(os.path.join(work_dir, "cp2k-v_hartree-1_0.cube"))
        self.eden_cube = Cp2kCube(os.path.join(work_dir, "cp2k-ELECTRON_DENSITY-1_0.cube"))
        self.totden_cube = Cp2kCube(os.path.join(work_dir, "cp2k-TOTAL_DENSITY-1_0.cube"))
    
    @property
    def data(self):
        data_dict = {}
        data_dict.update(self.stdout.energy_dict)
        data_dict["energy"] = self.stdout.energy
        data_dict["force"] = self.stdout.force
        data_dict["hartree_cube"] = self.hartree_cube.cube_data
        data_dict["eden_cube"] = self.eden_cube.cube_data
        data_dict["totden_cube"] = self.totden_cube.cube_data
        return data_dict

In [7]:
task_hpc = Cp2kTest("hpc")

In [8]:
task_custom = Cp2kTest("custom")

In [23]:
for k, v in task_hpc.data.items():
    print("-" * 50)
    print(k)
    print("-" * 50)
    if isinstance(v, float):
        print("hpc: ", v)
        print("custom: ", task_custom.data[k])
        print("difference: ", task_custom.data[k] - v) 
    elif isinstance(v, list) and len(v) == 1:
        print("hpc: ", v[0])
        print("custom: ", task_custom.data[k][0])
        print("difference: ", task_custom.data[k][0] - v[0])
    else:
        hpc_data = np.reshape(v, -1)
        custom_data = np.reshape(task_custom.data[k], -1)
        print("RMSE: ", np.sqrt(np.mean((hpc_data - custom_data) ** 2)))
    print("-" * 50)

--------------------------------------------------
Total energy
--------------------------------------------------
hpc:  -7672.85412780205
custom:  -7672.855110313824
difference:  -0.0009825117740547284
--------------------------------------------------
--------------------------------------------------
Overlap energy of the core charge distribution
--------------------------------------------------
hpc:  1.2291773931934016e-06
custom:  1.2291773931934016e-06
difference:  0.0
--------------------------------------------------
--------------------------------------------------
Self energy of the core charge distribution
--------------------------------------------------
hpc:  -20857.792994571886
custom:  -20857.792994571886
difference:  0.0
--------------------------------------------------
--------------------------------------------------
Core Hamiltonian energy
--------------------------------------------------
hpc:  4377.6846858133085
custom:  4377.684133903319
difference:  -0.00055